# **Generating the Word Embeddings for users across Fifteen Subreddits**

This notebook considers 15 subreddits. For each user in a subreddit, it calculates the mean word embedding from all his comments. Then it calculates it generates the cosine similarity of language use between user pairs.

This notebook uses the SentenceBert.


---

**Part 1: Reading the data**<br>
**Part 2: Generate the word embeddings** <br>
  Generate word embedding for every commnent, and then take the mean for every user. This embedding represents his language usage on the reddit platform.<br>
**Part 3: Find Cultural Similarity between user pairs**<br>
  For every user pair, find the cosine similarity between their word embeddings<br>

---

.


.


---
# **Part 1: Reading the data**

In this section, I have read the 15 subreddits.

---
.

Check if cuda is being used

In [1]:
import torch
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

Using cuda.


Connect to drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
data_fifteen_subreddits = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/reddit_project/data_fifteen_subreddits.csv', low_memory=False)
print(len(data_fifteen_subreddits)) #length of data = 107352
print(len(pd.unique(data_fifteen_subreddits['subreddit_id']))) #number of subreddits considered = 16 #but is 15
print(len(pd.unique(data_fifteen_subreddits['id']))) #unique number of comments = , the data is at the comment level =
print(len(pd.unique(data_fifteen_subreddits['parent_id']))) #number of parent nodes =
print(len(pd.unique(data_fifteen_subreddits['link_id']))) #number of submissions =
print(len(data_fifteen_subreddits.columns))
print(pd.unique(data_fifteen_subreddits['subreddit_id'])) #there are fifteen unique subreddits, the nana rows are ignored

107352
16
107351
50325
6156
17
['t5_22i0' 't5_2r2jt' 't5_3deqz' 't5_2sjgc' 't5_2scss' 't5_2r4oc'
 't5_2wm0g' 't5_2qmpb' 't5_2vbli' 't5_2qhwp' 't5_2qh33' 't5_2ror6'
 't5_2sgoq' 't5_2qm35' 't5_2qo4s' nan]


---
# **Part 2: Generate the word embeddings**

In this section, I have read the 15 subreddits. Generate word embedding for every commnent, and then take the mean for every user. This embedding represents his language usage on the reddit platform.

---
.

Import libraries

In [4]:
!pip install -U sentence-transformers
# load tqdm
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=86e441177b8298b4637832c7c678d3b26c0ac2a2204cfa8318efa0074efd8686
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     - 91.8 kB 4.8 MB/s 0:00:00
  Preparing metadata (setup.py) ... 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 10.1 MB/s eta 0:00:00
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=c4693021547437cfdc4ea2d331806fd1e92d6e4654e814396326b053e73d5d0a
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [5]:
import numpy as np
import pandas as pd
import nltk
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample, losses, util
from torch.utils.data import DataLoader
from sentence_transformers import losses
import os
import swifter
from nltk.tokenize import sent_tokenize
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

print data

In [6]:
print(len(data_fifteen_subreddits))
data_fifteen_subreddits.head(3)

107352


,edited,id,parent_id,distinguished,created_utc,author_flair_text,author_flair_css_class,controversiality,subreddit_id,retrieved_on,link_id,author,score,gilded,stickied,body,subreddit
0,0,dbumnpz,t1_dbulzrw,NaN,1.483229e+09,NaN,NYAN,0.0,t5_22i0,1.485680e+09,t3_5lc6zb,captnkaposzta,2.0,0.0,False,Beileid? Kiwi Fernsehgarten Trinkspiele retten...,de
1,0,dbumnq0,t1_dbum9w2,NaN,1.483229e+09,NaN,NaN,0.0,t5_2r2jt,1.485680e+09,t3_5lai4x,CampyJejuni,3.0,0.0,False,Wrong subreddit mate.,TwoXChromosomes
2,0,dbumnq1,t3_5lb9zs,NaN,1.483229e+09,NaN,NaN,0.0,t5_3deqz,1.485680e+09,t3_5lb9zs,Luigimario280,7.0,0.0,False,Karma!,BikiniBottomTwitter


clean up the data

First take only the 'author' and 'body' columns

In [8]:
data_word_embeddings = data_fifteen_subreddits[['id','author','body']]
print(len(data_word_embeddings)) #length of data = 107352
print(len(pd.unique(data_word_embeddings['id'])))
print(len(pd.unique(data_word_embeddings['author']))) #number of authors = 39443
print(data_word_embeddings.columns) #only two columns

107352
107351
39443
Index(['id', 'author', 'body'], dtype='object')


drop comments with missing body

In [9]:
data_word_embeddings = data_word_embeddings.dropna(subset=['body'])
print(len(data_word_embeddings))
data_word_embeddings.head(3)

107345


,id,author,body
0,dbumnpz,captnkaposzta,Beileid? Kiwi Fernsehgarten Trinkspiele retten...
1,dbumnq0,CampyJejuni,Wrong subreddit mate.
2,dbumnq1,Luigimario280,Karma!


delete those comments which have body as '[deleted]'. We see that there are 5849 such rows.

In [10]:
data_word_embeddings[data_word_embeddings['body'] == '[deleted]']

,id,author,body
6,dbumnq5,[deleted],[deleted]
19,dbumntx,[deleted],[deleted]
22,dbumnvw,[deleted],[deleted]
41,dbumomw,[deleted],[deleted]
53,dbump65,[deleted],[deleted]
...,...,...,...
107292,dbx5oqk,[deleted],[deleted]
107313,dbx5p56,[deleted],[deleted]
107333,dbx5pm6,[deleted],[deleted]
107340,dbx5ppu,[deleted],[deleted]


In [11]:
data_word_embeddings = data_word_embeddings[data_word_embeddings['body'] != '[deleted]']
print(len(data_word_embeddings))
print(len(pd.unique(data_word_embeddings['author']))) #number of authors = 39440
print(data_word_embeddings.head(3))

101496
39440
        id         author                                               body
0  dbumnpz  captnkaposzta  Beileid? Kiwi Fernsehgarten Trinkspiele retten...
1  dbumnq0    CampyJejuni                              Wrong subreddit mate.
2  dbumnq1  Luigimario280                                             Karma!


using a pretrained SBERT model to encode the sentences rather than training on the reddit data itself

In [12]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('all-mpnet-base-v2')

Took around 20 min on a A100 gpu

In [13]:
embeddings_list = []
j = 0

for ind, row in data_word_embeddings.iterrows():
  j += 1
  if j % 5000 == 0:
    print('created embedding for '+str(ind)+'/'+str(len(data_word_embeddings))+' comments')

  curr_comment = row['body']

  sentence_embedding = sbert_model.encode(curr_comment)
  embeddings_list.append(sentence_embedding)

data_word_embeddings['word_embedding'] = embeddings_list

print(len(data_word_embeddings))
print(data_word_embeddings.head(3))

created embedding for 5267/101496 comments
created embedding for 10548/101496 comments
created embedding for 15862/101496 comments
created embedding for 21176/101496 comments
created embedding for 26505/101496 comments
created embedding for 31782/101496 comments
created embedding for 37031/101496 comments
created embedding for 42308/101496 comments
created embedding for 47586/101496 comments
created embedding for 52891/101496 comments
created embedding for 58238/101496 comments
created embedding for 63541/101496 comments
created embedding for 68839/101496 comments
created embedding for 74113/101496 comments
created embedding for 79383/101496 comments
created embedding for 84625/101496 comments
created embedding for 89949/101496 comments
created embedding for 95212/101496 comments
created embedding for 100488/101496 comments
created embedding for 105764/101496 comments
101496
        id         author                                               body  \
0  dbumnpz  captnkaposzta  Beile

note: the index above goes till 107 thousand, but there are only 101496 comments considered

In [14]:
#print the length of the first row embedding
print(data_word_embeddings['word_embedding'][0])
print(len(data_word_embeddings['word_embedding'][0])) #length is 768

[-1.91892609e-02 -1.07530758e-01  3.59330177e-02  7.45008811e-02
 -6.39176667e-02 -9.34633706e-03 -1.27879996e-02  1.14389695e-02
  3.55925895e-02 -1.67494081e-02 -1.23001458e-02 -1.36922654e-02
  5.78009873e-04  9.22742300e-03 -1.65713094e-02 -9.56412330e-02
  2.16582939e-02 -3.83907892e-02  1.46209449e-02 -1.48299448e-02
 -3.42881270e-02  1.46549474e-03  5.41099021e-03  7.02678366e-03
 -1.15189170e-02 -5.93410805e-03 -2.29337346e-02 -1.54325385e-02
  6.70398176e-02  2.20283289e-02  4.74969931e-02 -4.27323096e-02
 -1.49366409e-02 -9.01708566e-03  1.62199399e-06 -1.06794173e-02
  1.80718973e-02 -2.30717044e-02 -1.14243403e-02  2.70166621e-02
 -3.73758301e-02  2.33756825e-02 -3.65488492e-02 -1.26677081e-02
  1.48021476e-02  5.65259717e-02 -4.38819937e-02  1.68793648e-02
 -3.63511555e-02  5.02629764e-02  1.49275195e-02 -3.19193071e-03
 -2.62589697e-02 -8.30725022e-03  4.75294925e-02 -5.62383495e-02
 -2.63304226e-02 -2.57369895e-02 -1.44834807e-02  2.02532802e-02
 -1.23237949e-02 -1.89951

Save the word embeddings for every author in a csv

Aggregate based on authors to have a single embedding for every author

In [18]:
#number of unique authors
print(len(data_word_embeddings)) #length of data embeddings data
print(len(pd.unique(data_word_embeddings['author']))) #number of authors = 39440

101496
39440


ignore. Run the below code only to find the mean embedding of each author. This vector would represent his average language usage on the entire reddit platform.

In [16]:
author_embeddings = data_word_embeddings.groupby(['author'], as_index=False)['word_embedding'].mean()
print(len(author_embeddings)) #length of data embeddings data
print(len(pd.unique(author_embeddings['author']))) #number of authors = 39440

39440
39440


In [17]:
print(len(author_embeddings))
print(author_embeddings.head(3))

39440
                 author                                     word_embedding
0                --AJ--  [0.0014906613, 0.013069552, 0.0020544275, -0.0...
1       --Hello_World--  [-0.04436073, -0.055472687, 0.014411808, -0.04...
2  --IIII--------IIII--  [-0.009615915, 0.08618535, -0.03001394, -0.065...


In [ ]:
author_embeddings.to_csv('/content/gdrive/MyDrive/Colab Notebooks/reddit_project/word_embeddings_fifteen_subreddits.csv')

---
# **Part 3: Find Cultural Similarity between user pairs**

For every comment, find the cosine similarity between their word embeddings

---
.

In [20]:
cul_sim_results = pd.DataFrame(columns=['subreddit_id','id','from_user','to_user','cultural_similarity'])
print(cul_sim_results)

Empty DataFrame
Columns: [subreddit_id, id, from_user, to_user, cultural_similarity]
Index: []


function to find cosine similarity

In [19]:
import numpy as np
def cosine_sim(vector1, vector2):
    return min(1., np.dot(vector1, vector2) / (np.linalg.norm(vector1, ord=2) * np.linalg.norm(vector2, ord=2)))

In [21]:
def cultural_similarity_function(input_data):

  ignore_comments_counter = 0
  j = 0

  #additional code to resolve an error
  type_base = type(input_data['parent_id'].iloc[0])

  for ind, row in input_data.iterrows():
    j += 1
    if j % 10000 == 0:
      print('finished comment '+str(j)+'/'+str(len(input_data)))

    curr_comment_id = row['id']
    curr_author = row['author']
    curr_subreddit_id = row['subreddit_id']
    curr_id = row['id']
    if type(row['parent_id']) != type_base:
      ignore_comments_counter
      continue
    curr_parent_comment_id = row['parent_id'][3:] #noticed that the parent id is nothing but the comment id preceded by 3 characters

    #find the parent comment
    #identify if a comment is a primary comment
    primary_comment_flag = 0
    if row['parent_id'] == row['link_id']: #it is a primary comment
      #print("primary comment")
      primary_comment_flag = 1
    if(len(input_data[input_data['id'] == curr_parent_comment_id]['author']) == 0): #the parent comment could not be found
      ignore_comments_counter += 1
      continue
    primary_comment_flag = 0
    curr_parent = input_data[input_data['id'] == curr_parent_comment_id]['author'].values[0]

    #find the word embeddings of sender and reciver and hence cosine similarity
    if len(data_word_embeddings[data_word_embeddings['id'] == curr_comment_id]['word_embedding'].values) == 0: #could not find the embedding of the sender
      ignore_comments_counter += 1
      continue
    else:
      from_user_embedding = data_word_embeddings[data_word_embeddings['id'] == curr_comment_id]['word_embedding'].values[0]

    if len(data_word_embeddings[data_word_embeddings['id'] == curr_parent_comment_id]['word_embedding'].values) == 0: #could not find the embedding of the receiver
      ignore_comments_counter += 1
      continue
    else:
      to_user_embedding = data_word_embeddings[data_word_embeddings['id'] == curr_parent_comment_id]['word_embedding'].values[0]

    if j == 0:
      print(from_user_embedding.shape)
      print(to_user_embedding.shape)
    cultsim = cosine_sim(from_user_embedding, to_user_embedding)

    cul_sim_results.loc[len(cul_sim_results.index)] = [curr_subreddit_id, curr_id, curr_author, curr_parent, cultsim]

  print('total number of comments ignored: ' +str(ignore_comments_counter))
  return cul_sim_results

data_culsim_user_pairs = cultural_similarity_function(data_fifteen_subreddits)

finished comment 10000/107352
finished comment 20000/107352
finished comment 30000/107352
finished comment 40000/107352
finished comment 50000/107352
finished comment 60000/107352
finished comment 70000/107352
finished comment 80000/107352
finished comment 90000/107352
finished comment 100000/107352
total number of comments ignored: 49662


at this stage, we have the cultural similarity for every comment (the cosine similarity of embeddings between the sender comment and the reciever comment). However, we actually want the *average* cosine similarity between the given user pair. SO we now take the mean across user pairs to obtain an average cosine similarity between the given user pair. (We will then feed this into the comment data set, and each comment will be associated with one average cosine similarity value). The average value is important because for each comment, we want the average cosine similarities (not just the cosine similarity representing that one comment) between the sender and reciever.

In [24]:
print(len(data_culsim_user_pairs))
print(data_culsim_user_pairs.head(3))

57685
  subreddit_id       id     from_user        to_user  cultural_similarity
0     t5_2qo4s  dbumpbm      Exoguana  passiveparrot             0.514737
1     t5_2qo4s  dbumphm       Stankie       yungtito             0.147589
2      t5_22i0  dbumqp3  SpaceHippoDE    oldandgreat             0.369992


confirm that user pairs are repeated

In [22]:
data_culsim_user_pairs.duplicated(subset=['from_user', 'to_user']).sum()

7713

In [23]:
data_culsim_user_pairs.to_csv('/content/gdrive/MyDrive/Colab Notebooks/reddit_project/data_fifteen_subreddits_comment_level_culsim.csv')

In [27]:
data_culsim_user_pairs = data_culsim_user_pairs.groupby(['from_user','to_user'], as_index=False)['cultural_similarity'].mean()

In [29]:
print(len(data_culsim_user_pairs))
print(data_culsim_user_pairs.head(3))

49972
  from_user         to_user  cultural_similarity
0    --AJ--          --AJ--             0.150673
1    --AJ--      Switch72nd             0.505629
2   --Nylon  tastefulchrist             0.134080


Confirm that there are no repeated user pairs

In [28]:
data_culsim_user_pairs.duplicated(subset=['from_user', 'to_user']).sum()

0

In [30]:
data_culsim_user_pairs.to_csv('/content/gdrive/MyDrive/Colab Notebooks/reddit_project/data_fifteen_subreddits_user_pair_level_culsim.csv')